In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from category_encoders import BinaryEncoder, HashingEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

Antes de empezar con el preprocesamiento de los datos, se seleccionan aquellas variables de interés para aplicar al modelo.

In [2]:
sanctionlist = pd.read_csv('datos_entrada_script_04_y_PowerBI/sanctionlist.csv').iloc[:,1:]
sanctionlist

,Name,EntityType,Address,Country,Origin,City
0,AEROCARIBBEAN AIRLINES,ENTERPRISE,NaN,CUBA,EEUU,HAVANA
1,"ANGLO-CARIBBEAN CO., LTD.",ENTERPRISE,"IBEX HOUSE, THE MINORIES",UNITED KINGDOM,EEUU,LONDON
2,BANCO NACIONAL DE CUBA,ENTERPRISE,ZWEIERSTRASSE 35,SWITZERLAND,EEUU,ZURICH
3,BANCO NACIONAL DE CUBA,ENTERPRISE,AVENIDA DE CONCHA ESPINA 8,SPAIN,EEUU,MADRID
4,BANCO NACIONAL DE CUBA,ENTERPRISE,"DAI-ICHI BLDG. 6TH FLOOR, 10-2 NIHOMBASHI, 2-C...",JAPAN,EEUU,TOKYO
...,...,...,...,...,...,...
25822,VIKTOR YANUKOVYCH,PERSON,NaN,NaN,UE,NaN
25823,VIKTOR FEDOROVICH YANUKOVYCH,PERSON,NaN,NaN,UE,NaN
25824,YEVHEN VITALIIOVYCH BALYTSKIY,PERSON,NaN,NaN,UE,NaN
25825,YEVGENIY VITALIEVICH BALYTSKIY,PERSON,NaN,NaN,UE,NaN


In [21]:
sanctionlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25827 entries, 0 to 25826
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        25827 non-null  object
 1   EntityType  25827 non-null  object
 2   Address     8230 non-null   object
 3   Country     14789 non-null  object
 4   Origin      25827 non-null  object
 5   City        12681 non-null  object
dtypes: object(6)
memory usage: 1.2+ MB


In [7]:
sanctionlist.describe()

,Name,EntityType,Address,Country,Origin,City
count,25827,25827,8230,14789,25827,12681
unique,20801,5,7099,190,2,2159
top,AERO CONTINENTE S.A,ENTERPRISE,"EAST SHAHID ATEFI STREET 35, AFRICA BOULEVARD,...",RUSSIA,EEUU,TEHRAN
freq,47,10225,30,1718,15258,986


A partir del resultado de describe() se puede estudiar la cardinalidad de cada variable:

In [18]:
for i in sanctionlist.describe().columns:
    print("La variable", i, "presenta un", round(sanctionlist.describe().loc['unique',i]/sanctionlist.describe().loc['count',i]*100,2),'% de valores únicos sobre el total de valores no nulos.\n')

La variable Name presenta un 80.54 % de valores únicos sobre el total de valores no nulos.

La variable EntityType presenta un 0.02 % de valores únicos sobre el total de valores no nulos.

La variable Address presenta un 86.26 % de valores únicos sobre el total de valores no nulos.

La variable Country presenta un 1.28 % de valores únicos sobre el total de valores no nulos.

La variable Origin presenta un 0.01 % de valores únicos sobre el total de valores no nulos.

La variable City presenta un 17.03 % de valores únicos sobre el total de valores no nulos.



A mayor porcentaje, mayor es la cardinalidad. No interesan las variables con un porcentaje de valores únicos alto porque esto podría acarrear problema de rendimiento y de calidad del modelo. Por lo tanto, se eliminarán las variables 'Name' y 'Address' del preprocesamiento.

In [3]:
ml_ns = sanctionlist.drop(columns=['Name','Address'])
ml_ns

,EntityType,Country,Origin,City
0,ENTERPRISE,CUBA,EEUU,HAVANA
1,ENTERPRISE,UNITED KINGDOM,EEUU,LONDON
2,ENTERPRISE,SWITZERLAND,EEUU,ZURICH
3,ENTERPRISE,SPAIN,EEUU,MADRID
4,ENTERPRISE,JAPAN,EEUU,TOKYO
...,...,...,...,...
25822,PERSON,NaN,UE,NaN
25823,PERSON,NaN,UE,NaN
25824,PERSON,NaN,UE,NaN
25825,PERSON,NaN,UE,NaN


Todas las variables son categóricas nominales. Hay que sustituirlas por valores numéricos por diversos métodos de codificación.

- Variable 'Origin': Como solo presenta dos valores únicos, la información se divide en dos columnas. (explicar como funciona codificación one-hot. Referencia: Libro)

- Variable 'EntityType': Presenta 5 valores únicos. Una codificación one-hot generaría 5 nuevas columnas. Para evitar crear tantas columnas se usa codificación binaria. (Explicar como funciona. Referencia: https://elmundodelosdatos.com/tecnicas-para-codificar-variables-categoricas-binaria-hashing/ )

- Variables 'Country' y 'Origin': Presentan muchos valores únicos. Con el objetivo de poder seleccionar el número de columnas que se van a generar, se aplica un algoritmo de hashing. (Explicar como funciona. Referencia: https://elmundodelosdatos.com/tecnicas-para-codificar-variables-categoricas-binaria-hashing/ )

Para poder codificar todas las variables, cada una con su método, en una sola línea de código se utiliza la clase ColumnTransformer de la librería skalearn.compose. ColumnsTransformer funciona como Pipeline pero pudiendo seleccionar la/s columna/s sobre las que se aplica la transformación.

## Modelo 1

De momento consta de 3 partes:

- Un imputador para poner a los valores nulos el valor más frecuente.
- Un codificador que codifica a binario todas las columnas. Cada una de ellas según el método más apropiado.
- Un PCA que permite reducir la dimensionalidad de las variables ajustando el parámetro n_components.
- Falta añadir el modelo de clustering (mirar modelos del libro: KMeans etc.)

Se añade un bucle for para aplicar distintos parámetros.

In [31]:
imputer = Pipeline([('imputer',SimpleImputer(strategy='most_frequent'))])
ml_ns_imp = imputer.fit_transform(ml_ns)

encoder = ColumnTransformer([
                            ("one-hot",OneHotEncoder(),['Origin']),
                            ("binary",BinaryEncoder(),['EntityType']),
                            ("hashing_country",HashingEncoder(drop_invariant=True),['Country']),
                            ("hashing_city",HashingEncoder(drop_invariant=True),['City'])
                         
])

preprocessing = Pipeline([('encoder',encoder),('pca',PCA())])

preprocessing.fit_transform(pd.DataFrame(ml_ns_imp,columns=ml_ns.columns))

array([[ 6.54703532e-01, -8.49166586e-01,  1.34565509e-01, ...,
        -1.45888528e-14, -5.42299862e-14,  7.54248971e-16],
       [ 6.75811570e-01, -8.99319148e-01, -9.84634061e-02, ...,
        -2.41888234e-14, -4.08792380e-14,  5.06637466e-17],
       [ 3.03281171e-01, -4.09796601e-01, -2.91481931e-01, ...,
        -1.25420672e-14, -6.08792932e-14,  7.38729707e-17],
       ...,
       [-1.31188949e+00,  2.05718912e-01, -2.28479844e-02, ...,
        -1.19203794e-16, -2.27336916e-16, -1.83011146e-17],
       [-1.31188949e+00,  2.05718912e-01, -2.28479844e-02, ...,
        -1.40753667e-16, -2.24705870e-16, -1.82940509e-17],
       [-1.31188949e+00,  2.05718912e-01, -2.28479844e-02, ...,
        -1.40753667e-16, -2.24705870e-16, -1.82940509e-17]])

In [5]:
encoder = ColumnTransformer([
                            ("one-hot",OneHotEncoder(),['Origin']),
                            ("binary",BinaryEncoder(),['EntityType']),
                            ("hashing_country",HashingEncoder(drop_invariant=True),['Country']),
                            ("hashing_city",HashingEncoder(drop_invariant=True,n_components=16),['City']), # Aumentamos el número de columnas para evitar colisiones de hash
                         
])


preprocessing = Pipeline([
                        ('encoder',encoder),
                        ('pca',PCA(n_components=0.85)), # Se preserva en 85% de la varianza inicial
                        ('std',)
                        ('Kmeans',)
])

preprocessing.fit_transform(ml_ns)

array([[ 6.60790556e-01, -7.40216832e-01,  1.64197994e-01, ...,
        -2.30152025e-13,  4.75661569e-14,  5.02604127e-16],
       [ 6.95727334e-01, -8.18456032e-01,  1.45247187e-01, ...,
         8.49080148e-15, -3.71712717e-14,  3.48425190e-17],
       [ 7.32204147e-01, -7.68834806e-01,  4.10860290e-01, ...,
        -7.66238328e-14,  1.21681899e-13, -2.37469021e-16],
       ...,
       [-1.35059956e+00,  1.68020709e-01, -1.15505227e-02, ...,
         1.47987327e-17, -1.95881758e-16, -2.00049108e-17],
       [-1.35059956e+00,  1.68020709e-01, -1.15505227e-02, ...,
         1.81077888e-17, -2.02591435e-16, -1.99935755e-17],
       [-1.35059956e+00,  1.68020709e-01, -1.15505227e-02, ...,
         1.81077888e-17, -2.02591435e-16, -1.99935755e-17]])

Se pueden hacer dos datasets para el modelo: uno en el que se eliminen outliers aplicando algún modelo de preprocesamiento y otro en el que se deje que sea el modelo ML el que detecte estas anomalías si es que las considera como tal. Se pueden hacer gráficas de codo y dendogramas.

Ideas para nuevos modelos: Eliminar imputador y dejar que se encargue el codificador. Añadir standarScaler o similar tras el PCA. Hacer un único hashing para City y Country

In [5]:
for i in range(8,40,8):

    encoder = ColumnTransformer([
                                ("one-hot",OneHotEncoder(),['Origin']),
                                ("binary",BinaryEncoder(),['EntityType']),
                                ("hashing_country",HashingEncoder(drop_invariant=True,n_components=i),['Country']),
                                ("hashing_city",HashingEncoder(drop_invariant=True,n_components=i),['City']), # Aumentamos el número de columnas para evitar colisiones de hash
                         
    ])

    encoded_data = encoder.fit_transform(ml_ns)

KeyboardInterrupt: 

In [5]:
encoder = ColumnTransformer([
                            ("one-hot",OneHotEncoder(),['Origin']),
                            ("binary",BinaryEncoder(),['EntityType']),
                            ("hashing_country",HashingEncoder(drop_invariant=True),['Country']),
                            ("hashing_city",HashingEncoder(drop_invariant=True,n_components=16),['City']), # Aumentamos el número de columnas para evitar colisiones de hash
                         
])
encoded_data = encoder.fit_transform(ml_ns)

In [34]:
hashing = HashingEncoder(drop_invariant=True)
trans_city = hashing.fit_transform(ml_ns['City'])

In [35]:
trans_city['concat'] = pd.Series(trans_city.fillna('').values.tolist()).map(lambda x: ''.join(map(str,x)))

In [36]:
trans_city

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,concat
0,0,0,0,0,0,0,1,0,00000010
1,0,0,0,0,0,1,0,0,00000100
2,0,0,0,0,1,0,0,0,00001000
3,0,0,0,1,0,0,0,0,00010000
4,0,0,0,0,1,0,0,0,00001000
...,...,...,...,...,...,...,...,...,...
25822,1,0,0,0,0,0,0,0,10000000
25823,1,0,0,0,0,0,0,0,10000000
25824,1,0,0,0,0,0,0,0,10000000
25825,1,0,0,0,0,0,0,0,10000000


In [37]:
len(trans_city['concat'].unique())

8